---
title: "SeeClick"
author: "Samith Va"
date: "2024-08-20"
categories: [LVLMs]
format:
    html:
        toc: true 
        code-fold: true
        code-summary: "Show the code"
---


**SeeClick** is a visual based GUI agent which is built on top of Qwen-VL-Chat. Using this agent, you can interact with the eletronics device in more efficient way, in other word it is a smarter agent (compare with Siri, Alexa, Google Assistant).  

# Model Architecture

How SeeClick is built? There are mainly 2 steps in forming SeeClick.

1. GUI-Grouding : Use many GUI datasets^[train with 1 millions dataset, contained 30 different task] to improve model understanding in the world of widgets such as [RICO](http://www.interactionmining.org/rico.html)

2. Fine-tuning : Fine-tune with goal-oriented instruction.

In simple words, There are two stages of finetuning, and each stage is finetuned with different dataset.

```{mermaid}
flowchart LR
  A[Qwen-VL-Chat-Int4] -->|Widget Caption Dataset| B[Qwen Checkpoint]
  B --> |AITW Dataset| C[SeeClick]
```


Due to limited resources, I only able to train the GUI-Grouding stage with only one small dataset ^[RICO, Widget Captioning] on 4 RX4090 GPUs with 24GB RAM each. Using q-LORA for the fine-tuning stage, it takes about 3 hours. Below is the training configuration :


```{bash}
torchrun $DISTRIBUTED_ARGS qwen_finetune.py \
    --model_name_or_path $MODEL \
    --data_path $DATA \
    --fp16 True \
    --fix_vit True \
    --output_dir output_qwen \
    --num_train_epochs 1 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 2 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1000 \
    --save_total_limit 10 \
    --learning_rate 1e-5 \
    --weight_decay 0.1 \
    --adam_beta2 0.95 \
    --warmup_ratio 0.01 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --report_to "none" \
    --model_max_length 512 \
    --lazy_preprocess True \
    --use_lora \
    --q_lora \
    --gradient_checkpointing \
    --deepspeed finetune/ds_config_zero2.json

```